# Лабораторная работа 3 "Разметка данных"

## Постановка задачи

- Возьмите датасет в первом дз
- Напишите ТЗ(в свободной форме) для разметки выбранного датасета. (Для текстового датасета можно предложить разметку семантики, для набора изображений – детекцию или сегментацию). Не забывайте про примеры в ТЗ!
- Добавьте какой-либо валидатор (принимаются подмешивание тестовых кейсов/рассчет метрики согласованности – в этом случае нужно будет предостаивть разметку от минимум 2х людей).
- Отправьте ваш собранный (но неразмеченный!) датасет знакомому с курса  и предложите разметить по вашему ТЗ. **Получите обратную связь.**
Для разметки допустимо использовать одну из онлайн-платформ или самостоятельные инструменты (предоставьте инструмент вашему разметчику).
- Проанализируйте полученные результаты разметки на наличие выбросов/дубликатов/некорректных результатов. Примените методы для их коррекции или удаления. Оцените метрику согласованности, либо квалификацию вашего разметчика на подмешанных тестовых кейсах.
- Порефлексируйте на тему процесса разметки данных и качества полученных резул итатов.менно для вашего случая.

## Описание датасета

**Идея:**

Получить данные о продуктах и ценах на них в разных магазинах, для возможности поиска наиболее выгодных предложений.
Также предполагается извлечение дополнительных признаков из описания товаров и поддержка гибкого добавления новых источников (магазинов).
Производится веб-скрейпинг данных с сайтов https://magnit.ru/ (Магнит) и https://5ka.ru/ (Пятерочка). В данной работе производится сбор данных о ценах на товары категории "Сыр", за продолжительное время (в идеале, за каждый день).

## Разметка

Есть выбор между двумя вариантами разметки:
- разметка бренда (поскольку функция для извлечения бренда из ЛР1 реализована не универсальной и ошибается)
- разметка типа сыра (твердый, полутвердый, рассольный и т.д.) по известным данным (жирность, is_creamy, категория - Бри, Пармезан и т.д.)

Первая разметка более простая и не требует экспертизы в области сыров. Второй вариант предполагает бОльшую сложность для разметчика и может допускать субъективность, однако, выглядит как более понятная для модели машинного обучения задача.

В этой работе попробуем разметить бренды сыра для возможности дальнейшего обучения модели с NER (https://www.kaggle.com/code/eneszvo/ner-named-entity-recognition-tutorial).

## Техническое задание

Разметить бренды для каждого уникального наименования сыра. Пример:

| Название                                                    | Бренд             |
|-------------------------------------------------------------|-------------------|
| Сыр плавленый Viola Сливочный 35% БЗМЖ 130г                 | Viola             |
| Плавленый продукт Рязанский ЗПС колбасный копченый 45% 300г | Рязанский ЗПС     |
| Сыр Сернурский СЗ Сернурская Легенда 45% 300г               | Сернурский СЗ     | 
| Сыр Брест-Литовск Тильзитер нарезка 45% 130г                | Брест-Литовск     |
| Сыр М Свежесть Косичка копченая 40% 100г                    | М Свежесть        |
| Сыр творожный Савушкин Воздушный сливочный 60% 150г         | Савушкин          |
| Сыр плавленый Карат Дружба 45% 90г                          | Карат             |
| Сыр Premiere of taste мягкий с белой плесенью Камамбер 125г | Premiere of taste |
| Сыр Магнит Белорусское золото 45% весовой                   | Магнит            |
| Сыр Мега Мастер Чечил Боровский 45% 100г 60% 150г"          | Мега Мастер       |

## Работа разметчика

Для разметки брендов была предоставлена выборка из датасета: набор уникальных брендов. Используемая платформа для разметки - Google Sheets (https://docs.google.com/spreadsheets/d/1OGNTovui8utSQ92NAzAKQFLr5Dk_0AjxBfuSQJlBy-A/edit?usp=sharing)

## Валидатор

Для валидации разметим некоторое количество записей (около 5-10%). 

## Импорт библиотек

In [40]:
!pip install -q psycopg2 matplotlib seaborn polars scikit-learn rapidfuzz

In [41]:
from sqlalchemy import create_engine, Column, Float, String, Date, Boolean, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

from typing import Optional, Tuple, Dict
import matplotlib.pyplot as plt
from datetime import date
import seaborn as sns
import pandas as pd
import polars as pl
import numpy as np
import requests
import random
import time
import csv
import os
import re

## Конфигурация подключения к базе данных

In [42]:
Base = declarative_base()

DATABASE_URL = "postgresql+psycopg2://{username}:{password}@{host}:{port}/{db_name}".format(
    host=os.getenv("POSTGRES_HOST", "localhost"),
    port=os.getenv("POSTGRES_PORT", "5432"),
    db_name=os.getenv("POSTGRES_DB", "store_parser"),
    username=os.getenv("POSTGRES_USER", "postgres"),
    password=os.getenv("POSTGRES_PASSWORD", "123456"),

)

def init_db():
    engine = create_engine(DATABASE_URL)
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    return Session()

C:\Users\admin\AppData\Local\Temp\ipykernel_20612\988954521.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


## Класс, соответствующий структуре таблицы в БД

In [43]:
class ProductFilled(Base):
    __tablename__ = 'store_products_filled'

    name = Column(String, nullable=False, primary_key=True)
    brand = Column(String)
    price = Column(Float)  
    weight = Column(Float)
    unit = Column(String)
    fat_content = Column(Float, nullable=True)
    is_sliced = Column(Boolean, nullable=True)
    is_bzmj = Column(Boolean, nullable=True)
    is_creamy = Column(Boolean, nullable=True)
    discount = Column(Float, nullable=True) 
    rating = Column(Float, nullable=True)
    date_added = Column(Date, default=date.today, primary_key=True)  
    store = Column(String, nullable=False, primary_key=True)

    __table_args__ = {'extend_existing': True}

    def __repr__(self):
        return f"<ProductFilled(name='{self.name}', price={self.price}, date_added={self.date_added}, store={self.store})>"

## Инициализация БД

In [48]:
session = init_db()

In [111]:
# Определение схемы данных
load_schema = {
    "name": pl.Utf8,
    "price": pl.Float64,
    "date_added": pl.Date,
    "store": pl.Utf8,
    "brand_auto": pl.Utf8,
    "weight": pl.Float32,
    "unit": pl.Utf8,
    "fat_content": pl.Float32,
    "is_sliced": pl.Boolean,
    "is_bzmj": pl.Boolean,
    "is_creamy": pl.Boolean,
    "discount": pl.Float32,
    "rating": pl.Float32
}

## Получение всех данных

In [113]:
results = (
    session
    .query(ProductFilled)
    .order_by(ProductFilled.date_added.asc())
    .all()
)

data = [{
    "name": product.name,
    "price": product.price,
    "date_added": product.date_added,
    "store": product.store,
    "brand_auto": product.brand,
    "weight": product.weight,
    "unit": product.unit,
    "fat_content": product.fat_content,
    "is_sliced": product.is_sliced,
    "is_creamy": product.is_creamy,
    "is_bzmj": product.is_bzmj,
    "discount": product.discount,
    "rating": product.rating
} for product in results]

# df = pd.DataFrame(data)
df = pl.DataFrame(data, schema=load_schema)
df.sample(5)

name,price,date_added,store,brand_auto,weight,unit,fat_content,is_sliced,is_bzmj,is_creamy,discount,rating
str,f64,date,str,str,f32,str,f32,bool,bool,bool,f32,f32
"""Продукт рассольный Сиртаки Ori…",149.99,2025-04-03,"""Магнит""","""Сиртаки Original""",200.0,"""г""",55.0,false,false,false,0.0,4.8
"""Сыр Hochland Сливочный плавлен…",359.99,2025-03-28,"""Пятерочка""","""Hochland Сливочный""",400.0,"""г""",55.0,false,true,true,10.0,4.94
"""Сыр Сыробогатов Дружба плавлен…",71.49,2025-04-04,"""Пятерочка""","""Сыробогатов Дружба""",80.0,"""г""",50.0,false,true,true,0.0,4.79
"""Сыр Landkaas Тильзитер 45% вес…",230.97,2025-03-31,"""Магнит""","""Landkaas Тильзитер""",1.0,"""кг""",45.0,false,false,false,10.0,4.6
"""Сыр ЭкоНива Momente aus Scholl…",199.99,2025-04-05,"""Магнит""","""ЭкоНива Momente""",200.0,"""г""",50.0,false,false,false,13.0,4.7


In [24]:
session.close()

## Выборочная ручная разметка

Разметим некоторые записи для валидации разметки

In [164]:
df = df.with_columns(
     pl.when(pl.col("name").str.to_lowercase().str.contains("viola"))
      .then(pl.lit("Viola"))  # Используем pl.lit для строкового значения
      .when(pl.col("name").str.to_lowercase().str.contains("kesidis"))
      .then(pl.lit("Kesidis"))
      .otherwise(None)
    .alias("brand_mark") 
)

In [166]:
(
    df.select(
        (1 - pl.col("brand_mark").is_null().mean()).alias("fill_percentage")
    ).item() * 100
)

6.6793893129771025

## Проверка результатов разметки

In [168]:
schema = {
    "name": pl.Utf8,
    "brand": pl.Utf8
}

In [170]:
df_marked = pl.read_csv("Сыр_разметка.csv", separator=';', schema=schema)

In [196]:
df_marked = df_marked.with_columns(
    pl.col("brand").str.strip_chars().alias('brand')
)

In [198]:
df_marked.head(10)

name,brand
str,str
"""Сыр Брест-Литовск Классический…","""Брест-Литовск"""
"""Сыр творожный Савушкин Воздушн…","""Савушкин"""
"""Сыр Viola Маасдам полутвердый …","""Viola"""
"""Сыр Сыробогатов Швейцарский 45…","""Сыробогатов"""
"""Сыр Galbani Моцарелла 45% БЗМЖ…","""Galbani"""
"""Сыр плавленый Карат Дружба 45%…","""Карат"""
"""Сыр Сваля Тильзитер нарезка 45…","""Сваля"""
"""Сыр мягкий Ришелье Бри с белой…","""Ришелье"""
"""Сыр Предгорье Кавказа Ассорти …","""Предгорье Кавказа"""


### Джойним с исходным датасетом

In [200]:
df_joined = (
    df
    .join(df_marked, on='name', how='left')
)
df_joined.sample(5)

name,price,date_added,store,brand_auto,weight,unit,fat_content,is_sliced,is_bzmj,is_creamy,discount,rating,brand_marked,brand_mark,brand
str,f64,date,str,str,f32,str,f32,bool,bool,bool,f32,f32,str,str,str
"""Сыр Предгорье Кавказа Ассорти …",169.99,2025-04-02,"""Магнит""","""Предгорье Кавказа Ассорти""",110.0,"""г""",45.0,false,false,false,0.0,4.5,null,null,"""Предгорье Кавказа"""
"""Сыр плавленый Карат Дружба 45%…",63.99,2025-04-03,"""Магнит""","""Карат Дружба""",90.0,"""г""",45.0,false,false,true,0.0,4.7,null,null,"""Карат"""
"""Сыр Gruyere твердый 49% БЗМЖ""",329.9,2025-03-28,"""Пятерочка""","""Gruyere""",1.0,"""кг""",49.0,false,true,false,0.0,4.79,null,null,"""нет"""
"""Сыр Поставы Городок Пармезан г…",179.99,2025-03-31,"""Магнит""","""Поставы Городок Пармезан""",200.0,"""г""",45.0,false,false,false,10.0,4.5,null,null,"""Поставы Городок"""
"""Сыр Sernur cheese Халумис из к…",174.99,2025-03-26,"""Пятерочка""","""Sernur""",200.0,"""г""",50.0,false,true,false,0.0,4.805882,null,null,"""Sernur cheese"""


In [202]:
# Количество уникальных брендов
count = (
    df_joined
    .select(pl.col('brand').unique())
    .filter(pl.col('brand').is_not_null()) 
    .height 
)
count

74

In [204]:
# Наиболее популярные бренды
top = (
    df_joined
    .unique(subset=['name', 'brand'])
    .group_by('brand')
    .agg(
        pl.len().alias('count')
    )
    .sort('count', descending=True)
)

top.head(10)

brand,count
str,u32
"""Hochland""",22
"""Брест-Литовск""",17
"""Viola""",14
"""Liebendorf""",13
"""Pretto""",7
"""Сыробогатов""",6
"""Вкус & Польза""",6
"""Экомилк""",6
"""Almette""",5


## Расчет точности ручной разметки по валидационной выборке

In [212]:
unique_brands = df_joined.select(["brand", "brand_mark"]).unique()

# Считаем совпадения среди уникальных брендов
matches = unique_brands.filter(
    (pl.col("brand") == pl.col("brand_mark")) & 
    (pl.col("brand_mark").is_not_null())
).height

# Общее количество размеченных уникальных брендов
total = unique_brands.filter(
    pl.col("brand_mark").is_not_null()
).height

print(f"Точность ручной разметки (по уникальным брендам): {matches/total:.1%} ({matches}/{total})")

Точность ручной разметки (по уникальным брендам): 100.0% (2/2)


## Сверка ручной точности и автоматической

In [254]:
unique_brands = df_joined.select(["brand", "brand_auto"]).unique()

matches_auto = unique_brands.filter(
    (pl.col("brand") == pl.col("brand_auto")) & 
    (pl.col("brand").is_not_null())
).height

total = unique_brands.filter(pl.col("brand").is_not_null()).height

print(f"Точность автоматической разметки: {matches_auto/total:.1%} ({matches_auto}/{total})")

Точность автоматической разметки: 17.0% (27/159)


In [258]:
# Примеры ошибок

unique_brands = df_joined.select(["brand", "brand_auto"]).unique()
    
errors = unique_brands.filter(
    (pl.col("brand") != pl.col("brand_auto")) & 
    (pl.col("brand").is_not_null())
)

if errors.height > 0:
    print("\nПримеры ошибок:")
    print(errors.select(["brand", "brand_auto"]))


Примеры ошибок:
shape: (132, 2)
┌───────────────────┬─────────────────────────────────┐
│ brand             ┆ brand_auto                      │
│ ---               ┆ ---                             │
│ str               ┆ str                             │
╞═══════════════════╪═════════════════════════════════╡
│ Viola             ┆ Viola Классический              │
│ нет               ┆ Дружба                          │
│ Предгорье Кавказа ┆ Предгорье Кавказа Чечил-спагет… │
│ Брест-Литовск     ┆ Брест-Литовск Королевский       │
│ ЭкоНива           ┆ ЭкоНива Momente                 │
│ …                 ┆ …                               │
│ Умалат            ┆ Умалат Сулугуни                 │
│ Брест-Литовск     ┆ Брест-Литовск Тильзитер         │
│ Радость вкуса     ┆ Радость                         │
│ President         ┆ President Сливочный             │
│ Карат             ┆ Карат Дружба                    │
└───────────────────┴─────────────────────────────────┘


## Выводы

**Что получилось:**
- Ручная разметка значительно улучшила покрытие брендов. Поскольку написание функции для парсинга и поиска бренда по наименованию не дала успешных результатов (точность ), необходимо использование модели типа NRE (например) для точного заполнения признака.
- Проведено сравнение результатов ручной разметки и валидационной выборки (точность разметчика: 100%). Ввиду относительной простоты задачи, ошибок не было допущено
- Проведено сравнение результатов ручной разметки человеком и автоматической (из ЛР1 парсинг по полю name). Приняв ручную разметку за истину, получили точность автоматической разметки как: 17%.

**Мысли:**
- Как вариант разметки дополнительной можно рассмотреть разметку категории сыра (твердый, мягкий, полутвердый и т.д)

**Применение:**

- Анализ ценовой политики по брендам
- Построение модели для прогнозирования спроса

Финальный вывод: Разметка качественная, но можно рассмотреть другие варианты ТЗ для разметки с новыми целями.окрытия данных.